Дорогой студент!

В домашнем задании Lite вам предлагается поработать подробнее с параметрами словаря и формированием гиперпараметров нейронной сети. Создайте 9 нейросетей с различными гиперпараметрами (см. пунтк 2 и 3)

 Для этого необходимо:

  1. Воссоздать ноутбук, аналогичный ноутбуку практической части №1, загрузив при этом необходимую нам базу (код уже доступен в ноутбуке).

  2. Задать в ноутбуке следующие параметры для размера словаря, ширины окна и шага:

    - Размер словаря - от 10000 до 20000 (выбрать меньшее значение диапазона, если будет перегрузка ОЗУ и перезапуск подключения к Colaboratory)
    - Ширина окна - от 1000 до 2000
    - Шаг - от 100 до 500 (на обучение лучше влияет наименьший шаг, но это может перегрузить ОЗУ).

  3. Создать архитектуру сети и задать гиперпараметры. Можно воспользоваться шаблоном:
  
   - Добавьте модель прямого распространения **Sequential()**
   - Добавьте один или несколько полносвязных (**Dense**) слоёв
   - Добавьте слои **Dropout()** и **BatchNormalization()**
   - Добавьте выходной полносвязный слой с количеством нейронов, соответствующим количеству классов (число писателей)
  
   Напомним, что точность сети можно проверить по значению показателя 'val_accuracy' на конце каждой эпохи.
   

In [2]:
# Загрузка датасетов из облака google
import gdown

In [3]:
# Загрузка датасета из облака
gdown.download('https://storage.yandexcloud.net/aiueducation/Content/base/l7/writers.zip', None, quiet=True)

'writers.zip'

In [4]:
# Распаковка архива в папку writers
!unzip -qo writers.zip -d writers/

# Просмотр содержимого папки
!ls writers

'(Булгаков) Обучающая_5 вместе.txt'
'(Булгаков) Тестовая_2 вместе.txt'
'(Клиффорд_Саймак) Обучающая_5 вместе.txt'
'(Клиффорд_Саймак) Тестовая_2 вместе.txt'
'(Макс Фрай) Обучающая_5 вместе.txt'
'(Макс Фрай) Тестовая_2 вместе.txt'
'(О. Генри) Обучающая_50 вместе.txt'
'(О. Генри) Тестовая_20 вместе.txt'
'(Рэй Брэдберри) Обучающая_22 вместе.txt'
'(Рэй Брэдберри) Тестовая_8 вместе.txt'
'(Стругацкие) Обучающая_5 вместе.txt'
'(Стругацкие) Тестовая_2 вместе.txt'


In [5]:
# Ваше задание здесь
# Работа с массивами данных
import numpy as np
from tensorflow.keras import utils
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import gdown
import os
import time
import re
import pickle
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
# Настройка констант для загрузки данных
FILE_DIR = "writers"  # Папка с текстовыми файлами
SIG_TRAIN = "обучающая"  # Признак обучающей выборки в имени файла
SIG_TEST = "тестовая"  # Признак тестовой выборки в имени файла

In [7]:
# Загрузка датасета. Добавляются имена классов и соответствующие тексты.
# Все тексты преобразуются в строку и объединяются для каждого класса и выборки
CLASS_LIST = []
text_train = []
text_test = []

for file_name in os.listdir(FILE_DIR):
    # Выделение имени класса и типа выборки из имени файла
    m = re.match(r"\((.+)\) (\S+)_", file_name)
    # Если выделение получилось, то файл обрабатывается
    if m:
        class_name = m[1]
        subset_name = m[2].lower()
        # Проверка типа выборки в имени файла
        is_train = SIG_TRAIN in subset_name
        is_test = SIG_TEST in subset_name

        # Если тип выборки обучающая либо тестовая - файл обрабатывается
        if is_train or is_test:
            # Добавление нового класса, если его еще нет в списке
            if class_name not in CLASS_LIST:
                print(f'Добавление класса "{class_name}"')
                CLASS_LIST.append(class_name)
                # Инициализация соответствующих классу строк текста
                text_train.append("")
                text_test.append("")

            # Поиск индекса класса для добавления содержимого файла в выборку
            cls = CLASS_LIST.index(class_name)
            print(
                f'Добавление файла "{file_name}" в класс "{CLASS_LIST[cls]}", {subset_name} выборка.'
            )
            with open(f"{FILE_DIR}/{file_name}", "r") as f:
                # Загрузка содержимого файла в строку
                text = f.read()
            # Определение выборки, куда будет добавлено содержимое
            subset = text_train if is_train else text_test
            # Добавление текста к соответствующей выборке класса. Концы строк заменяются на пробел
            subset[cls] += " " + text.replace("\n", " ")

# Определение количества классов
CLASS_COUNT = len(CLASS_LIST)

Добавление класса "Стругацкие"
Добавление файла "(Стругацкие) Обучающая_5 вместе.txt" в класс "Стругацкие", обучающая выборка.
Добавление класса "О. Генри"
Добавление файла "(О. Генри) Обучающая_50 вместе.txt" в класс "О. Генри", обучающая выборка.
Добавление класса "Булгаков"
Добавление файла "(Булгаков) Тестовая_2 вместе.txt" в класс "Булгаков", тестовая выборка.
Добавление класса "Рэй Брэдберри"
Добавление файла "(Рэй Брэдберри) Обучающая_22 вместе.txt" в класс "Рэй Брэдберри", обучающая выборка.
Добавление класса "Клиффорд_Саймак"
Добавление файла "(Клиффорд_Саймак) Тестовая_2 вместе.txt" в класс "Клиффорд_Саймак", тестовая выборка.
Добавление файла "(Стругацкие) Тестовая_2 вместе.txt" в класс "Стругацкие", тестовая выборка.
Добавление файла "(Клиффорд_Саймак) Обучающая_5 вместе.txt" в класс "Клиффорд_Саймак", обучающая выборка.
Добавление класса "Макс Фрай"
Добавление файла "(Макс Фрай) Обучающая_5 вместе.txt" в класс "Макс Фрай", обучающая выборка.
Добавление файла "(О. Генри) Те

In [8]:
# Диапазоны параметров
vocab_sizes = [10000, 15000, 20000]
win_sizes = [1000, 1500, 2000]
win_hops = [100, 300, 500]

combinations = [
    (10000, 1000, 100),
    (15000, 1500, 300),
    (20000, 2000, 500),
    (10000, 1500, 500),
    (15000, 2000, 100),
    (20000, 1000, 300),
    (10000, 2000, 300),
    (15000, 1000, 500),
    (20000, 1500, 100)
]

In [9]:
models = []

for i, (vocab_size, win_size, win_hop) in enumerate(combinations, 1):
    print(f"\n=== Модель {i}: vocab_size={vocab_size}, win_size={win_size}, win_hop={win_hop} ===")

    # Токенизация и создание словаря
    tokenizer = Tokenizer(num_words=vocab_size,
                         filters='!"#$%&()*+,-–—./…:;<=>?@[\\]^_`{|}~«»\t\n\xa0\ufeff',
                         lower=True, split=' ',
                         oov_token='неизвестное_слово',
                         char_level=False)
    tokenizer.fit_on_texts(text_train)

    # Преобразование текстов в последовательности
    seq_train = tokenizer.texts_to_sequences(text_train)
    seq_test = tokenizer.texts_to_sequences(text_test)

    # Функции для создания окон
    def split_sequence(sequence, win_size, hop):
        return [sequence[i:i + win_size] for i in range(0, len(sequence) - win_size + 1, hop)]

    def vectorize_sequence(seq_list, win_size, hop):
        class_count = len(seq_list)
        x, y = [], []
        for cls in range(class_count):
            vectors = split_sequence(seq_list[cls], win_size, hop)
            x += vectors
            y += [utils.to_categorical(cls, class_count)] * len(vectors)
        return np.array(x), np.array(y)
    # Создание выборок
    x_train, y_train = vectorize_sequence(seq_train, win_size, win_hop)
    x_test, y_test = vectorize_sequence(seq_test, win_size, win_hop)

    # Преобразование в Bag of Words
    x_train_bow = tokenizer.sequences_to_matrix(x_train.tolist())
    x_test_bow = tokenizer.sequences_to_matrix(x_test.tolist())

    # Создание модели
    model = Sequential()
    model.add(Dense(200, input_dim=vocab_size, activation="relu"))
    model.add(Dropout(0.25))
    model.add(BatchNormalization())
    model.add(Dense(CLASS_COUNT, activation='softmax'))

    # Компиляция модели
    model.compile(optimizer='adam',
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])

    # Обучение модели
    print(f"Обучение модели {i}...")
    history = model.fit(x_train_bow, y_train,
                       epochs=20,
                       batch_size=128,
                       validation_data=(x_test_bow, y_test),
                       verbose=1)

    # Сохранение результатов
    models.append({
        'model': model,
        'history': history,
        'params': {
            'vocab_size': vocab_size,
            'win_size': win_size,
            'win_hop': win_hop
        },
        'train_acc': history.history['accuracy'][-1],
        'val_acc': history.history['val_accuracy'][-1]
    })



=== Модель 1: vocab_size=10000, win_size=1000, win_hop=100 ===


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Обучение модели 1...
Epoch 1/20
138/138 ━━━━━━━━━━━━━━━━━━━━ 7s 30ms/step - accuracy: 0.9224 - loss: 0.2369 - val_accuracy: 0.8693 - val_loss: 0.4125
Epoch 2/20
138/138 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 1.0000 - loss: 8.8336e-04 - val_accuracy: 0.8714 - val_loss: 0.3634
Epoch 3/20
138/138 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 1.0000 - loss: 3.9591e-04 - val_accuracy: 0.8703 - val_loss: 0.3538
Epoch 4/20
138/138 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 1.0000 - loss: 2.4833e-04 - val_accuracy: 0.8693 - val_loss: 0.3541
Epoch 5/20
138/138 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 1.0000 - loss: 1.7916e-04 - val_accuracy: 0.8715 - val_loss: 0.3475
Epoch 6/20
138/138 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 1.0000 - loss: 1.2905e-04 - val_accuracy: 0.8724 - val_loss: 0.3445
Epoch 7/20
138/138 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 1.0000 - loss: 1.0760e-04 - val_accuracy: 0.8730 - val_loss: 0.3421
Epoch 8/20
138/138 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accu

In [10]:
# Вывод результатов
for i, m in enumerate(models, 1):
    print(f"Модель {i}:")
    print(f"  Параметры: vocab_size={m['params']['vocab_size']}, win_size={m['params']['win_size']}, win_hop={m['params']['win_hop']}")
    print(f"  Точность на обучении: {m['train_acc']:.4f}")
    print(f"  Точность на валидации: {m['val_acc']:.4f}")
    print()

Модель 1:
  Параметры: vocab_size=10000, win_size=1000, win_hop=100
  Точность на обучении: 1.0000
  Точность на валидации: 0.8851

Модель 2:
  Параметры: vocab_size=15000, win_size=1500, win_hop=300
  Точность на обучении: 1.0000
  Точность на валидации: 0.9086

Модель 3:
  Параметры: vocab_size=20000, win_size=2000, win_hop=500
  Точность на обучении: 1.0000
  Точность на валидации: 0.9142

Модель 4:
  Параметры: vocab_size=10000, win_size=1500, win_hop=500
  Точность на обучении: 1.0000
  Точность на валидации: 0.8988

Модель 5:
  Параметры: vocab_size=15000, win_size=2000, win_hop=100
  Точность на обучении: 1.0000
  Точность на валидации: 0.9333

Модель 6:
  Параметры: vocab_size=20000, win_size=1000, win_hop=300
  Точность на обучении: 1.0000
  Точность на валидации: 0.8861

Модель 7:
  Параметры: vocab_size=10000, win_size=2000, win_hop=300
  Точность на обучении: 1.0000
  Точность на валидации: 0.9299

Модель 8:
  Параметры: vocab_size=15000, win_size=1000, win_hop=500
  Точнос